In [4]:
import os

In [6]:
os.chdir(r'C:\Users\Administrator\Desktop\abi_chatbot\specialized_chatbot')

In [8]:
!pip install -r reqirement.txt

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Administrator\\Desktop\\abi_chatbot\\specialized_chatbot\\.venv\\Lib\\site-packages\\pandas\\tests\\window\\test_expanding.py'
Check the permissions.




  Using cached llama_index-0.4.33-py3-none-any.whl
  Using cached langchain-0.0.117-py3-none-any.whl (414 kB)
  Using cached openai-0.27.2-py3-none-any.whl (70 kB)
  Using cached wikipedia-1.4.0-py3-none-any.whl
  Using cached unstructured-0.5.4-py3-none-any.whl
  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
  Using cached docx2txt-0.8-py3-none-any.whl
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached aiohttp-3.8.4-cp311-cp311-win_amd64.whl (317 kB)
  Using cached dataclasses_json-0.5.7-py3-none-any.whl (25 kB)
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
  Using cached pandas-1.5.3-cp311-cp311-win_amd64.whl (10.3 MB)
  Using cached tiktoken-0.3.2-cp311-cp311-win_amd64.whl (578 kB)
  Using cached SQLAlchemy-1.4.47-cp311-cp311-win_amd64.whl (1.6 MB)
  Using cached pydantic-1.10.6-cp311-cp311-win_amd64.whl (2.1 MB)
  Using cached argilla-1.4.0-py3-none-any.whl (2.0 MB)
  Using cached lxml-4.9.2-cp311-cp311-win_amd64.whl (3.8 MB)
  Using cached nltk-3.8.

  Using cached llama_index-0.4.33.tar.gz (147 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached langchain-0.0.117-py3-none-any.whl (414 kB)
  Using cached openai-0.27.2-py3-none-any.whl (70 kB)
  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached unstructured-0.5.4.tar.gz (1.3 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached aiohttp-3.8.4-cp311-cp311-win_amd64.whl (317 kB)
  Using cached dataclasses_json-0.5.7-py3-none-any.whl (25 kB)
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
 

In [20]:
import json
config = json.load(open('../config.json'))
os.environ["OPENAI_API_KEY"] = config['openai_api_key']

In [9]:
import llama_index

In [11]:
from typing import Any, Dict, Optional, Sequence, Type, cast, Union
import logging
import json
import json
from langchain import OpenAI

from llama_index.indices.vector_store.base import GPTVectorStoreIndex
from llama_index.vector_stores import (
    ChromaVectorStore,
    FaissVectorStore,
    PineconeVectorStore,
    QdrantVectorStore,
    SimpleVectorStore,
    WeaviateVectorStore,
)
from llama_index import (
    GPTSimpleVectorIndex,
    SimpleDirectoryReader,
    WikipediaReader,
    GPTListIndex,
    GPTKeywordTableIndex,
    GPTSimpleKeywordTableIndex,
    LLMPredictor, GPTSimpleVectorIndex, PromptHelper,
    QuestionAnswerPrompt
)
from llama_index.indices.query.vector_store.queries import (
    GPTChromaIndexQuery,
    GPTFaissIndexQuery,
    GPTSimpleVectorIndexQuery,

)
from langchain.agents import initialize_agent, Tool

In [71]:
MODEL = "text-davinci-003"

LLM = OpenAI(
    temperature=0,
    model_name=MODEL,
    max_tokens=256
)

LLM_PREDICTOR = LLMPredictor(llm=LLM)

In [55]:
competitors = [
    "Heineken",
    "Molson Coors", 
    "Carlsberg", 
    "Constellation",
    "Pabst Brewing"
]

In [62]:
DEFAULT_TEXT_QA_PROMPT_TMPL = (
    "You are an expert to beer and be glad to recommand beer \n"
    "Given the context information and not prior knowledge, answer the question as trully as possible, if the question beer brands is not contained in the context, say 'sorry I don't know' \n"
    f"If your answer might contains anything about {','.join(competitors)}, please do say 'Sorry I don't know\n"
    "---------------------\n"
    "Context: {context_str}"
    "\n---------------------\n"
    "Please use the same language"
    "{query_str}"
)

DEFAULT_TEXT_QA_PROMPT = QuestionAnswerPrompt(DEFAULT_TEXT_QA_PROMPT_TMPL)

In [47]:
documents = SimpleDirectoryReader('../data').load_data()

In [48]:
bot = GPTSimpleVectorIndex(documents)

INFO:llama_index.token_counter.token_counter:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_documents] Total embedding token usage: 61763 tokens


In [66]:
r = bot.query('prompt:Recommand some beer from Heineken\nresponse:', text_qa_template = DEFAULT_TEXT_QA_PROMPT)
print(r)
print(r.source_nodes)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 618 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 14 tokens


Sorry I don't know
[SourceNode(source_text='Kwak provides following products: Kwak Amber, Kwak Blonde, Kwak Rouge\n\n\nBrand Introduction\nPauwel Kwak was a brewer and owner of the ‘De Hoorn’ Inn in Dendermonde at the time of Napoleon. He was also a very inventive innkeeper.\nDuring those times, coachmen who stopped at an inn were not allowed to leave their coach and horses in order to have their thirst quenched, unlike their passengers. So Pauwel Kwak came up with an ingenious solution. He commissioned a special Kwak glass to be blown, a glass that could be hung safely on a coach. Allowing any coachman who visited his inn to enjoy a Kwak glass of beer like everyone else. His invention was a runaway success.\nIn 1982 inspired by this tale, Ivo Bosteels created a special “Pauwel Kwak” amber beer and recreated his iconic glass design, so that the legacy of this unconventional genius continues to be enjoyed today.\nIn 2022 the Kwak range expanded and the original Kwak Amber beer was joine

In [64]:
r = bot.query('Question:你知道Heineken吗\nAnswer:', text_qa_template = DEFAULT_TEXT_QA_PROMPT)
print(r)
print(r.source_nodes)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1250 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 19 tokens


Sorry I don't know.
[SourceNode(source_text='-Introduction\n哈尔滨啤酒源自1900年。作为中国最早的啤酒，哈尔滨啤酒始终坚持品质至上的原则，使产品遍布全国，更远销全球30多个国家和地区。哈尔滨啤酒作为年轻人的标志性啤酒品牌，长久以来，通过嘻哈，运动，时尚，电竞等兴趣点和年轻人建立更深层的沟通，成为年轻群体最喜欢的啤酒品牌。\n2015年，哈尔滨啤酒入选BrandZ中国最有价值品牌100强、BrandZ成长最快啤酒品牌，品牌市场价值一直稳步增长。2019年哈尔滨啤酒在Brand Finance发布的“全球最具价值啤酒品牌”品牌价值排名上升一位至第四名，成为唯一跻身前五的中国本土品牌。\nHarbin Brewery was established in 1900 and was the oldest brewery brand in China. Harbin always prioritize brand quality and its products are sold all over the country and even more than 30 overseas countries and areas. As a famous beer brand for young people, Harbin Brewery has beeing building deeper relationship with the young through hip-hop, sports, fashion, esports and others, and now become the most popular brand among young people.\nIn 2015, Harbin Brewery was listed in the BrandZ top 100 most valuable brand in China and the fastest growing beer brand, also the market value was increasing stably. In 2019, Harbin Brewery was listed as the fourth most valuable 

In [65]:
r = bot.query('Recommand some beer from Heineken', text_qa_template = DEFAULT_TEXT_QA_PROMPT)
print(r)
print(r.source_nodes)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 632 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 8 tokens


,Molson Coors,Carlsberg,Constellation,Pabst Brewing
Sorry, I don't know.
[SourceNode(source_text='Kwak provides following products: Kwak Amber, Kwak Blonde, Kwak Rouge\n\n\nBrand Introduction\nPauwel Kwak was a brewer and owner of the ‘De Hoorn’ Inn in Dendermonde at the time of Napoleon. He was also a very inventive innkeeper.\nDuring those times, coachmen who stopped at an inn were not allowed to leave their coach and horses in order to have their thirst quenched, unlike their passengers. So Pauwel Kwak came up with an ingenious solution. He commissioned a special Kwak glass to be blown, a glass that could be hung safely on a coach. Allowing any coachman who visited his inn to enjoy a Kwak glass of beer like everyone else. His invention was a runaway success.\nIn 1982 inspired by this tale, Ivo Bosteels created a special “Pauwel Kwak” amber beer and recreated his iconic glass design, so that the legacy of this unconventional genius continues to be enjoyed today.\nIn 2022 the Kwak ran

In [67]:
r = bot.query('Question:Do you know Corona beer \nAnswer:', text_qa_template = DEFAULT_TEXT_QA_PROMPT)
print(r)
print(r.source_nodes)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 4208 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 13 tokens




Yes, Corona is a brand of beer produced by Mexican brewery Cervecería Modelo and owned by Belgian company AB InBev. It is the top-selling brand of imported beer in the United States and is also popular in Mexico, where it is the official beer of both the NASCAR PEAK Mexico and NASCAR Cup Series. It is often served with a wedge of lime or lemon in the neck of the bottle to add tartness and flavor. The recipe for the mash bill includes corn as well as the barley malt and hops traditionally used for making beer. The brand's most popular variation is Corona Extra, a pale lager. It is one of the top-selling beers worldwide, and Corona Extra has been the top-selling imported drink in the U.S. since 1998.Other variants of the Corona beer brand include Corona Light, Corona Premier, and Corona Familiar. A variety of flavored hard seltzers marketed under the Corona brand name was launched in March 2020.
[SourceNode(source_text='Summary\nCorona (beer)\nCorona is a brand of beer produced by Mexi

In [68]:
r = bot.query('Question:Recommand some beer from Heineken please \nAnswer:', text_qa_template = DEFAULT_TEXT_QA_PROMPT)
print(r)
print(r.source_nodes)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 620 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 15 tokens


 Sorry I don't know.
[SourceNode(source_text='Kwak provides following products: Kwak Amber, Kwak Blonde, Kwak Rouge\n\n\nBrand Introduction\nPauwel Kwak was a brewer and owner of the ‘De Hoorn’ Inn in Dendermonde at the time of Napoleon. He was also a very inventive innkeeper.\nDuring those times, coachmen who stopped at an inn were not allowed to leave their coach and horses in order to have their thirst quenched, unlike their passengers. So Pauwel Kwak came up with an ingenious solution. He commissioned a special Kwak glass to be blown, a glass that could be hung safely on a coach. Allowing any coachman who visited his inn to enjoy a Kwak glass of beer like everyone else. His invention was a runaway success.\nIn 1982 inspired by this tale, Ivo Bosteels created a special “Pauwel Kwak” amber beer and recreated his iconic glass design, so that the legacy of this unconventional genius continues to be enjoyed today.\nIn 2022 the Kwak range expanded and the original Kwak Amber beer was joi

In [ ]:
r = bot.query('Question:Recommand some beer from Heineken please \nAnswer:', text_qa_template = DEFAULT_TEXT_QA_PROMPT)
print(r)
print(r.source_nodes)

In [73]:
r = bot.query('Question:推荐几款Heineken \nAnswer:', text_qa_template = DEFAULT_TEXT_QA_PROMPT)
print(r)
print(r.source_nodes)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 4048 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 20 tokens



Sorry, I don't know the specific Heineken beers you are referring to, but some popular Heineken beers include Goose Island IPA, Goose Island Gagga Goose, and Punch Cat Mango Flip. They all have different flavor profiles and alcohol content, ranging from 5.9% vol. to 5.5% vol. and IBU levels from 14 to 55.
[SourceNode(source_text='百威旗下高端品牌产品包括：经典拉格，特色拉格，小麦艾尔，IPA，水果啤酒，其它艾尔，跨啤酒\n\n\n经典拉格\n科罗娜经典系列：诞生于墨西哥，品牌理念崇尚自然、提倡自信，倡导本真、轻松与向往，是世界前五大啤酒品牌之一。啤酒口感清醇，色彩明亮，与青柠风味相得益彰，适合在海滩、后院等地方饮用，酒精含量4.6%。\n时代啤酒：源于比利时鲁汶，具有六百余年历史。选用萨兹贵族酒花，带来轻柔果香，与酵母中的松露以及菠萝果香相得益彰，同时精选进口大麦 ，口感醇厚顺滑，呈现蛋蛋的谷物与麦芽风味，酒精度≥5%vol，原麦汁浓度11.3°P。\n三宝乐啤酒系列：源于北海道，具有百年历史，呈现100%日本札幌风味，酒体澄澈轻盈，口感清爽顺滑，适合佐餐配食，酒精度≥5%vol，原麦汁浓度11°P。\n蓝妹啤酒系列：于1892年起源于德国，采用分段式双温发酵工艺，麦芽于酒花香味浓郁，入口顺滑爽口，展现纯正的德国口感。啤酒酒精度≥4.5%vol，原麦汁浓度10.2°P。蓝妹啤酒曾连续14年蝉联香港进口啤酒销量第一，2019~2020年为国内销量第一的高端进口啤酒。\n拳击猫右勾拳：属于德式淡色拉格，产品酒体轻盈呈淡金色，入口清新干爽，给人以畅快之感；具有浓郁麦香，佐餐解渴且百搭。产品酒精浓度不小于4.5% vol.，苦度为15 IBU。\n\n\n特色拉格\n拳击猫第一血：属于琥珀拉格，曾屡次获奖，酿造时特别添加红枣，麦芽略带蜂蜜与焦糖的浓郁香气，入口清甜回甘，酒精浓度不小于4.5% vol.，苦度为15 IBU。\n鹅岛贪杯鹅：属于酒花

In [72]:
r = bot.query('Question:推荐几款茅台 \nAnswer:', text_qa_template = DEFAULT_TEXT_QA_PROMPT)
print(r)
print(r.source_nodes)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 4123 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 22 tokens




Sorry, I don't know. However, some popular Maotai brands include the Goose Island IPA, Goose Island Gagaga Goose, and Punch Cat Mango Flip. The Goose Island IPA has a citrus and hop flavor, with an alcohol content of 5.9% vol. and an IBU of 55. The Goose Island Gagaga Goose is a light IPA with West Coast and Simcoe hops, with an alcohol content of 4.7% vol. and an IBU of 20. The Punch Cat Mango Flip is a mango hazy IPA with a tropical fruit flavor and a beer-like texture, with an alcohol content of at least 5.5% vol. and an IBU of 40.
[SourceNode(source_text='百威旗下高端品牌产品包括：经典拉格，特色拉格，小麦艾尔，IPA，水果啤酒，其它艾尔，跨啤酒\n\n\n经典拉格\n科罗娜经典系列：诞生于墨西哥，品牌理念崇尚自然、提倡自信，倡导本真、轻松与向往，是世界前五大啤酒品牌之一。啤酒口感清醇，色彩明亮，与青柠风味相得益彰，适合在海滩、后院等地方饮用，酒精含量4.6%。\n时代啤酒：源于比利时鲁汶，具有六百余年历史。选用萨兹贵族酒花，带来轻柔果香，与酵母中的松露以及菠萝果香相得益彰，同时精选进口大麦 ，口感醇厚顺滑，呈现蛋蛋的谷物与麦芽风味，酒精度≥5%vol，原麦汁浓度11.3°P。\n三宝乐啤酒系列：源于北海道，具有百年历史，呈现100%日本札幌风味，酒体澄澈轻盈，口感清爽顺滑，适合佐餐配食，酒精度≥5%vol，原麦汁浓度11°P。\n蓝妹啤酒系列：于1892年起源于德国，采用分段式双温发酵工艺，麦芽于酒花香味浓郁，入口顺滑爽口，展现纯正的德国口感。啤酒酒精度≥4.5%vol，原麦汁浓度10.2°P。蓝妹啤酒

In [74]:
!pip install langid

     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
      --------------------------------------- 0.0/1.9 MB 163.8 kB/s eta 0:00:12
      --------------------------------------- 0.0/1.9 MB 164.3 kB/s eta 0:00:12
     - -------------------------------------- 0.1/1.9 MB 233.8 kB/s eta 0:00:08
     - -------------------------------------- 0.1/1.9 MB 286.7 kB/s eta 0:00:07
     - -------------------------------------- 0.1/1.9 MB 275.8 kB/s eta 0:00:07
     -- ------------------------------------- 0.1/1.9 MB 327.4 kB/s eta 0:00:06
     --- ------------------------------------ 0.2/1.9 MB 403.5 kB/s eta 0:00:05
     ---- ----------------------------------- 0.2/1.9 MB 492.1 kB/s eta 0:00:04
     

In [88]:
import langid
import json

with open("data/lang_code.json", 'r') as f:
    lang_code = json.load(f) 

def lang_detect(text:str)->str:
    """detect language
    Parameter:
    --------
        text: str
            the text for which the langudate you want to detect
        
    Return
    ------
        langudge
    """
    # Language detection
    try:
        code = langid.classify(text)[0]
        lang = lang_code[code]
    except Exception as e:
        print(e)
        lang = 'English'
    return lang
      

In [89]:
lang_detect("是")

'Chinese'

In [90]:
lang_code['zh']

'Chinese'

In [93]:
bot.llm_predictor.__dict__

{'_llm': OpenAI(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x0000023F436EE0D0>, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.0, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False),
 'retry_on_throttling': True,
 '_total_tokens_used': 38503,
 'flag': True,
 '_last_token_usage': 4048}

In [94]:
bot.lang_detect

AttributeError: 'GPTSimpleVectorIndex' object has no attribute 'lang_detect'